# Transformações dos Dados

In [ ]:
%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.express as px
import seaborn as sns
import import_ipynb
import services.lista_estados_regioes as ler
import services.visualization as view
import services.lista_estados_regioes as lesr

In [ ]:
dataset = pd.read_csv('dados/dataset_poscomp.csv')

## Adicionar os eixos e somar de cada candidato

In [ ]:
dataset['matematica'] = dataset.loc[:,['Matemática Discreta', 'Cálculo Diferencial e Integral', 'Álgebra Linear',
 'Geometria Analítica', 'Análise Combinatória', 'Probabilidade e Estatística', 'Lógica Matemática']].sum(axis=1)

In [ ]:
dataset['fund_computacao'] = dataset.loc[:,['Organização de Arquivos e Dados','Técnicas de Programação',
 'Circuitos Digitais', 'Análise de Algoritmos', 'Teoria dos Grafos', 'Linguagens Formais, Autômatos e Computabilidade',
 'Algoritmos e Estruturas de Dados', 'Linguagens de Programação', 'Sistemas Operacionais', 'Arquitetura e Organização de Computadores']].sum(axis=1)

In [ ]:
dataset['tec_computacao'] = dataset.loc[:,['Redes de Computadores', 'Computação Gráfica',
                                                   'Inteligência Artificial', 'Engenharia de Software', 'Sistemas Distribuídos',
                                                   'Compiladores', 'Processamento de Imagens', 'Banco de Dados']].sum(axis=1)

In [ ]:
dataset['total'] = dataset['matematica'] + dataset['fund_computacao'] + dataset['tec_computacao']

In [ ]:
dataset=dataset.drop_duplicates(subset='INSCRICAO', keep='first')
dataset['ANO'].value_counts()

In [ ]:
cargo = {'1-Mestrado/Doutorado': '1', '2-Autoavaliação': '2'}
dataset = dataset.replace({'CARGO': cargo}, {'ESTADO': ler.estados_n})
dataset = dataset.rename(columns={'PRESENTE DIA1': 'PRESENTE'})

In [ ]:
# dataset['ESTADO'] = dataset['ESTADO'].replace(ler.estados_n)
# dataset['SEXO'] = dataset['SEXO'].replace(['M','F'],[1,0])
dataset['CARGO'] = dataset['CARGO'].replace(['Mestrado/Doutorado','Autoavaliação'],[0,1])
# dataset['REGIAO'] = dataset['REGIAO'].replace(ler.siglas_n)

In [ ]:
#Aqui estou adicionando as regioes dos estados brasileiros
for i in lesr.regioes_brasil_siglas:
    dataset.loc[dataset['ESTADO'].isin(lesr.regioes_brasil_siglas[i]), 'REGIAO'] = i

In [ ]:
dados_poscomp = dataset.drop_duplicates(subset='INSCRICAO', keep='first')
dados_poscomp = dados_poscomp.set_index('INSCRICAO')

In [ ]:
estados_brasileiros = {0: 'AC',
           1: 'AL',
           2: 'AM',
           3: 'AP',
           4: 'BA',
           5: 'CE',
           6: 'DF',
           7: 'ES',
           8: 'GO',
           9: 'MA',
           10: 'MG',
           11: 'MS',
           12: 'MT',
           13: 'PA',
           14: 'PB',
           15: 'PE',
           16: 'PI',
           17: 'PR',
           18: 'RJ',
           19: 'RN',
           20: 'RO',
           21: 'RR',
           22: 'RS',
           23: 'SC',
           24: 'SP',
           25: 'SE',
           26: 'TO'}

In [ ]:
por_sexo_estado = dados_poscomp.groupby(['ESTADO', 'SEXO']).size().reset_index(name='candidatos')

por_sexo_estado['ESTADO'] = por_sexo_estado['ESTADO'].replace(estados_brasileiros)
por_sexo_estado['SEXO'] = por_sexo_estado['SEXO'].replace({0: 'Feminino', 1: 'Masculino'})

# por_sexo_estado

In [ ]:
view.barplot_view(dataframe=por_sexo_estado.sort_values(['candidatos'], ascending=False),
                  y='candidatos',
                  x='ESTADO',
                  hue='SEXO',
                  fontsize=18,
                 figsize=(18,10), 
                 title_legend='Sexo',
                 xlabel='Unidade Federativa',
                 ylabel='Quantidade de candidatos')

In [ ]:
counts_presente = dados_poscomp.groupby(['PRESENTE', 'ANO']).size().reset_index(name='counts')

counts_presente['PRESENTE'] = counts_presente['PRESENTE'].replace({'N': 'Ausente', 'S': 'Presente'})

view.barplot_view(counts_presente, x='ANO', y='counts', hue='PRESENTE', 
                  title_legend='Participação')

In [ ]:
counts_presente = dados_poscomp.groupby(['PRESENTE', 'ANO']).size().reset_index(name='counts')

counts_presente['PRESENTE'] = counts_presente['PRESENTE'].replace({'N': 'Ausente', 'S': 'Presente'})

view.barplot_view(counts_presente, x='ANO', y='counts', hue='PRESENTE', title_legend='Participação')

In [ ]:
dados_presentes = dados_poscomp.query('PRESENTE == "S"')

counts_presente = dados_presentes.groupby(['SEXO', 'ANO']).size().reset_index(name='counts')

counts_presente['SEXO'] = counts_presente['SEXO'].replace({0: 'Feminino', 1: 'Masculino'})

view.barplot_view(counts_presente, x='ANO', y='counts', hue='SEXO', title_legend='Sexo', figsize=(8,6))

In [ ]:
dados_faltantes = dados_poscomp.query('PRESENTE == "N"')

counts_faltantes = dados_faltantes.groupby(['SEXO', 'ANO']).size().reset_index(name='counts')

counts_faltantes['SEXO'] = counts_faltantes['SEXO'].replace({0: 'Feminino', 1: 'Masculino'})

view.barplot_view(counts_faltantes, x='ANO', y='counts', hue='SEXO', title_legend='Sexo', figsize=(8,6),
                 ylabel='Quantidade de candidatos', xlabel='Ano', save_path='imagens/grafico_porFaltantes.eps')

In [ ]:
view.plot_top_especialidades_por_sexo(dados_poscomp, 'M', top_n=10, save_path='imagens/espec_masculino.eps')
view.plot_top_especialidades_por_sexo(dados_poscomp, 'F', top_n=10, save_path='imagens/espec_feminino.eps')

In [ ]:
media_hist = dados_poscomp.query('PRESENTE == "S"')

In [ ]:
def plot_histogram(dataset, columns):
    for column in columns:
        sns.histplot(dataset[column], kde=True)
        plt.xlabel(f'Média de {column}')
        plt.ylabel('Frequência')
        plt.title(f'Distribuição das médias de {column}')
        plt.show()

columns = ['matematica', 'fund_computacao', 'tec_computacao']
plot_histogram(media_hist, columns)

In [ ]:
dados_poscomp = dados_poscomp.query('PRESENTE != "N"')
dados_poscomp.shape

In [ ]:
media = dados_poscomp['total'].mean()
desvio_padrao = dados_poscomp['total'].std()
variancia = dados_poscomp['total'].var()

In [ ]:
print(f'Média: {media}, Desvio Padrão: {desvio_padrao}, Variância: {variancia}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Probabilidade desejada
probabilidade = 0.90

# Encontre os valores de X correspondentes ao intervalo de probabilidade
limite_inferior, limite_superior = norm.interval(probabilidade, loc=media, scale=desvio_padrao)

# Valores para o gráfico
x = np.linspace(media - 3 * desvio_padrao, media + 3 * desvio_padrao, 1000)
pdf = norm.pdf(x, loc=media, scale=desvio_padrao)

# Plote o gráfico da PDF
plt.plot(x, pdf, label='PDF')
plt.fill_between(x, pdf, where=(x >= limite_inferior) & (x <= limite_superior), alpha=0.5, label=f'Área de {probabilidade*100}%')
plt.xlabel('Valores de X')
plt.ylabel('PDF')
plt.title(f'Distribuição Normal ({media}, {desvio_padrao})')

# Exiba o gráfico
plt.legend()
plt.grid(True)
plt.show()

# Exiba o intervalo de valores de X correspondentes à probabilidade
print(f'Intervalo de {probabilidade*100}% de probabilidade: [{limite_inferior:.2f}, {limite_superior:.2f}]')

In [ ]:
base = dados_poscomp[dados_poscomp['total'] <= 17]

plt.figure(figsize=(10, 6))
hist = base['ESPECIALIDADE'].value_counts().plot(kind='bar')

for i, v in enumerate(base['ESPECIALIDADE'].value_counts()):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.title('Histograma das Especialidades')
plt.xlabel('Especialidade')
plt.ylabel('Contagem')

plt.show()

In [ ]:
base.shape

In [ ]:
base['ESPECIALIDADE'].value_counts()[:5]

In [ ]:
view.histplot(dataframe=base, x='total', kde=True, bins=10)

In [ ]:
sns.boxplot(y=dados_poscomp["total"]);

#### Técnica para eliminar os outlier

In [ ]:
df = dados_poscomp.copy()

In [ ]:
def check_outlier(value, quantiles, amplitude):
    if (value < quantiles[0] - (1.5 * amplitude)) or (value > quantiles[2] + (1.5 * amplitude)):
        return True
    return False

In [ ]:
# quartis das quantidades de caracteres dos títulos
import statistics

quantiles_title = statistics.quantiles(df["total"])
print("Quartis de nota:", quantiles_title)

# amplitude interquartil das quantidades de caracteres dos títulos

amplitude_title = quantiles_title[2] - quantiles_title[0]
print("Amplitude interquartil de nota:", amplitude_title)

# determinação dos outliers nos títulos

df["outlier_nota"] = df["total"].map(lambda x: check_outlier(
    value=x,
    quantiles=quantiles_title,
    amplitude=amplitude_title
))

In [ ]:
sns.boxplot(y=df["total"]);

In [ ]:
df = df.loc[~((df["outlier_nota"]))]
df = df.iloc[:,:-1]
df.shape

In [ ]:
print(f'Perda - {(374 / 9849) * 100:.2f}% dos dados') # informação importante

In [ ]:
dados_poscomp.shape

In [ ]:
nova_dados_poscomp = dados_poscomp[dados_poscomp['total'] >= 17]

In [ ]:
base.shape, nova_dados_poscomp.shape

In [ ]:
nova_dados_poscomp = nova_dados_poscomp.drop(['PRESENTE','CARGO', 'ANO'], axis=1)

In [ ]:
df = df.drop(['PRESENTE','CARGO', 'ANO'], axis=1)

In [ ]:
nova_dados_poscomp.to_csv('dados/df_poscompTeste.csv', index=False)

## Verificar o comportamento dos candidatos de acordo com a linha de pesquisa

In [ ]:
df.head()

In [ ]:
nova_dados_poscomp['area_concentration'].value_counts()

In [ ]:
nova_dados_poscomp['linhas_pesquisas'].value_counts()

In [ ]:
# plt.figure(figsize=(20, 15))
# plot = sns.lmplot(x="total", y="total", data=dados_poscomp, fit_reg=False, hue='linhas_pesquisas', legend=False)

# # Move the legend to an empty part of the plot
# plt.legend(loc='lower right')
# plt.show()